In [57]:
import pandas as pd
import numpy as np
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows as df_to_row
from fuzzywuzzy import process, fuzz

In [48]:
from szp_funcs import print_df, get_otv, load_month, path_to_docs

In [43]:
month = 'may'
status = ['Увольнение']

In [20]:
res = load_month(month)
otv = get_otv()

In [38]:
res

,inn,snils,sum_may,job_may,day_may,type_may,stv_may,status_may,status_pref_may,wday_may,nday_may
0,5003021368,001-171-362 75,58500.00,Помощник воспитателя,1.0,Основное место работы,1.0,Работа,Работа,18.0,18.0
1,5003021368,001-191-281 85,112184.88,Учитель,1.0,Основное место работы,1.0,Работа,Работа,18.0,18.0
2,5003021368,001-244-107 65,118681.78,Учитель,0.0,Основное место работы,1.0,Работа,Работа,15.0,18.0
3,5003021368,001-263-234 77,79619.03,Воспитатель,0.0,Основное место работы,1.0,Отпуск основной,Работа,16.0,18.0
4,5003021368,001-322-594 80,37907.50,Специалист,0.0,Основное место работы,1.0,Работа,Работа,9.0,18.0
...,...,...,...,...,...,...,...,...,...,...,...
188387,9718071371,198-317-420 02,110851.00,Бухгалтер,1.0,Основное место работы,1.0,Работа,Работа,18.0,18.0
188388,9718071371,199-418-719 35,153851.00,Бухгалтер,1.0,Основное место работы,1.0,Работа,Работа,18.0,18.0
188389,9718071371,205-412-630 13,209199.25,Начальник группы,0.0,Основное место работы,1.0,Работа,Работа,13.0,18.0
188390,9718071371,206-135-584 35,27528.33,Бухгалтер,0.0,Основное место работы,1.0,NaN,NaN,6.0,11.0


In [44]:
ans = res[(res.status_may.isin(status) == False) & (res.type_may == 'Основное место работы') & (res.job_may == 'Учитель')][['inn', 'snils']].groupby('inn').nunique().reset_index()

In [46]:
ans['snils'].sum()

56583

In [36]:
otv

,inn,okr,otv,sch_name,ekis
0,7743085543,САО,Рагимова Виктория Олеговна,ГБОУ Спецшкола № 1,13964.0
1,7713229928,САО,Горбанева Светлана Васильевна,ГБОУ ПМКК,13136.0
2,7704271031,ЦАО,Майильян Елена Александровна,ГБПОУ ПТ № 2,13277.0
3,7714093099,САО,Горбанева Светлана Васильевна,ГКОУ СКОШИ № 2,13139.0
4,7709438340,ЦАО,Майильян Елена Александровна,ГБПОУ КСУ № 3,11681.0
...,...,...,...,...,...
935,7728113806,NaN,NaN,ИКИ РАН,NaN
936,7728013512,NaN,NaN,ИПУ РАН,NaN
937,7717067481,NaN,NaN,ГБОУ ШОР № 1,NaN
938,7729133548,NaN,NaN,СУНЦ МГУ,NaN


In [49]:
kadr = pd.read_excel(path_to_docs + 'ppl_with_status.xlsx')

In [64]:
def matcher(row, look_up, name):
    if pd.isna(row[name]):
        return np.nan
    ans = process.extractOne(row[name], look_up, scorer=fuzz.WRatio, score_cutoff=60)
    if ans:
        return ans[0]
    return ans

In [61]:
uniq_jobs = pd.DataFrame(kadr['job'].unique())
uniq_jobs.columns = ['job']

In [62]:
jobs = pd.read_excel(path_to_docs + 'input.xlsx', sheet_name='должности')
jobs.columns = ['pre_job', 'job_106']
look_up = jobs['pre_job'].to_list()

In [65]:
uniq_jobs['pre_job'] = uniq_jobs.apply(lambda row: matcher(row, look_up, 'job'), axis=1)

In [67]:
uniq_jobs = pd.merge(uniq_jobs, jobs, how='left')

In [69]:
kadr = pd.merge(kadr, uniq_jobs[['job', 'job_106']])

In [78]:
ans = pd.merge(ans, kadr[(kadr['type'] == 'Внешнее совместительство') & (kadr['job_106'] == 'Учитель')].groupby('inn')['snils'].nunique().reset_index().rename(columns= {'snils': 'vnesh'}), how='left')

In [73]:
kadr.status.unique()

array(['Работа', 'Отпуск по уходу за ребенком', 'Отпуск основной', nan,
       'Отпуск по беременности и родам', 'Болезнь',
       'Отпуск учебный оплачиваемый',
       'Отпуск неоплачиваемый по разрешению работодателя',
       'Работа в отпуске по уходу за ребенком', 'Командировка',
       'Отсутствие с сохранением оплаты', 'Дополнительный отпуск',
       'Болезнь без оплаты', 'Отсутствие по невыясненным причинам',
       'Увольнение', 'Простой, отстранение по вине работника',
       'Дополнительный отпуск (неоплачиваемый)',
       'Отпуск неоплачиваемый по законодательству',
       'Отпуск учебный неоплачиваемый',
       'Отпуск на санаторно-курортное лечение', 'Прогул'], dtype=object)

In [80]:
ans = pd.merge(ans, kadr[(kadr['status'].isin(['Отпуск по беременности и родам', 'Отпуск по уходу за ребенком'])) & (kadr['job_106'] == 'Учитель')].groupby('inn')['snils'].nunique().reset_index().rename(columns= {'snils': 'obr'}), how='left')

In [81]:
print_df(pd.merge(ans, otv[['inn', 'sch_name']], how='left'), 'Список учителей для МЭШ ' + month)